## VacationPy

## Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

City_ID         City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
0        0   malmberget  67.1753   20.6549    -36.69        68          95   
1        1   carutapera  -1.1950  -46.0200     24.96        94          99   
2        2     waitangi -43.9535 -176.5597     15.10        87          16   
3        3  yellowknife  62.4560 -114.3525    -28.10        86          75   
4        4       albany  42.6001  -73.9662      2.36        78         100   

   Wind Speed Country        Date  
0        0.00      SE  1704327037  
1        2.75      BR  1704327037  
2        6.05      NZ  1704327038  
3        4.12      CA  1704326746  
4        1.55      US  1704326812

## Step 1: Create a map that displays a point for every city in the city_data_df DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidity_map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.8,
    alpha = 0.5,
    color = "City"
)

# Display the map
humidity_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

## Step 2: Narrow down the city_data_df DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df['Max Temp'] > 21) &
                      (city_data_df['Max Temp'] < 30) &
                      (city_data_df['Humidity'] > 40) &
                      (city_data_df['Humidity'] < 60) &
                      (city_data_df['Wind Speed'] < 4.5)]

# Drop any rows with null values
ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

City_ID         City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
46        46       lisala   2.1500   21.5167     23.41        53          57   
70        70      kaiapoi -43.3783  172.6401     26.42        59          80   
87        87     vallenar -28.5708  -70.7581     22.88        51           0   
106      106    mount isa -20.7333  139.5000     29.87        54          70   
111      111      salalah  17.0151   54.0924     22.05        49           0   
215      215       goroka  -6.0834  145.3873     25.30        54          91   
241      241   san carlos   9.6612  -68.5827     28.30        56          78   
313      313      omaruru -21.4333   15.9333     21.95        52          31   
377      377       sokode   8.9833    1.1333     22.84        53          82   
389      389       arauca   7.0847  -70.7591     27.89        47          84   
399      399        outjo -20.1167   16.1500     23.28        59          79   
400      400  sao vicente -23.9631  -46.3919     26.26        54          80   
402      402       kukawa  12.9234   13.5606     21.04        50          81   

     Wind Speed Country        Date  
46         0.15      CD  1704327063  
70         2.57      NZ  1704327076  
87         2.36      CL  1704327092  
106        4.12      AU  1704327101  
111        1.54      OM  1704327103  
215        0.97      PG  1704327167  
241        1.93      VE  1704327179  
313        1.51     NaN  1704327223  
377        3.09      TG  1704327254  
389        0.95      CO  1704327241  
399        2.83     NaN  1704327264  
400        1.34      BR  1704327265  
402        4.05      NG  1704327266

## Step 3: Create a new DataFrame called hotel_df.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

City Country      Lat       Lng  Humidity Hotel Name
46        lisala      CD   2.1500   21.5167        53           
70       kaiapoi      NZ -43.3783  172.6401        59           
87      vallenar      CL -28.5708  -70.7581        51           
106    mount isa      AU -20.7333  139.5000        54           
111      salalah      OM  17.0151   54.0924        49           
215       goroka      PG  -6.0834  145.3873        54           
241   san carlos      VE   9.6612  -68.5827        56           
313      omaruru     NaN -21.4333   15.9333        52           
377       sokode      TG   8.9833    1.1333        53           
389       arauca      CO   7.0847  -70.7591        47           
399        outjo     NaN -20.1167   16.1500        59           
400  sao vicente      BR -23.9631  -46.3919        54           
402       kukawa      NG  12.9234   13.5606        50

## Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories":"accommodation.hotel",
          "apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
lisala - nearest hotel: No hotel found
kaiapoi - nearest hotel: No hotel found
vallenar - nearest hotel: Hotel del Marqués
mount isa - nearest hotel: Ibis Styles
salalah - nearest hotel: Muscat International Hotel
goroka - nearest hotel: Bird of Paradise Hotel
san carlos - nearest hotel: Hotel Villas del Norte
omaruru - nearest hotel: Central Hotel
sokode - nearest hotel: No hotel found
arauca - nearest hotel: Hotel Nova Park
outjo - nearest hotel: Ombinda Country Lodge
sao vicente - nearest hotel: Pousada Vitória
kukawa - nearest hotel: No hotel found


City Country      Lat       Lng  Humidity  \
46        lisala      CD   2.1500   21.5167        53   
70       kaiapoi      NZ -43.3783  172.6401        59   
87      vallenar      CL -28.5708  -70.7581        51   
106    mount isa      AU -20.7333  139.5000        54   
111      salalah      OM  17.0151   54.0924        49   
215       goroka      PG  -6.0834  145.3873        54   
241   san carlos      VE   9.6612  -68.5827        56   
313      omaruru     NaN -21.4333   15.9333        52   
377       sokode      TG   8.9833    1.1333        53   
389       arauca      CO   7.0847  -70.7591        47   
399        outjo     NaN -20.1167   16.1500        59   
400  sao vicente      BR -23.9631  -46.3919        54   
402       kukawa      NG  12.9234   13.5606        50   

                     Hotel Name  
46               No hotel found  
70               No hotel found  
87            Hotel del Marqués  
106                 Ibis Styles  
111  Muscat International Hotel  
215      Bird of Paradise Hotel  
241      Hotel Villas del Norte  
313               Central Hotel  
377              No hotel found  
389             Hotel Nova Park  
399       Ombinda Country Lodge  
400             Pousada Vitória  
402              No hotel found

## Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.8,
    alpha = 0.5,
    color = "City",
    hover_cols = ["Hotel Name", "Country"])

# Display the map
hotel_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)